## Sumário:
- EDA 
- Treinamento
- Validação

## Objetivo
NCJNet é um modelo preditivo que visa sugerir cursos para o usuário, a predição é feito sob análise de notas e preferência do usuário.

## Configuração do Ambiente
É necessário o ambiente virtual para o funcionamento, para isso, realize os seguintes comandos no terminal
- /NCJNet$ pipenv shell
- /NCJNet$ pipenv install

Após isso, o ambiente está configurado com todas as bibliotecas necessárias. Para verificar, compile o código a baixo.

In [90]:
#import matplotlib.pyplot as plt
#import numpy as np
#import pandas as pd
#import seaborn as sns
#from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Todas as bibliotecas são necessárias para a compilação

# EDA

In [91]:
import pandas as pd
import seaborn as sns

## Criar DataFrames dos Datasets 

In [92]:
# Carregar DataFrame de clientes (customers.csv)
df = pd.read_csv('./datasets/RegistroAcompanhamento.csv') # leitura dos dados do csv

# visualizar os dados dos 5 primeiros clientes
df.head(5)

,Identificador,"""Codigo externo do registro""",Etapa,"""Data da criacao""","""Momento do ultimo ganho""","""Momento da ultima perda""","""Na mesma etapa desde""","""Codigo - Oferta de curso""","""Nome - Oferta de curso""","""Identificador da pessoa""",...,Redacao,Quimica,Fisica,Biologia,Geografia,Historia,Filosofia,Sociologia,Artes,Curso
0,318,"""- - -""","""- - -""","""16/06/2025 20:02""","""- - -""","""- - -""","""- - -""","""- - -""","""- - -""",360,...,90,89,88,86,87,99,86,85,89,Biblioteconomia
1,317,"""- - -""","""- - -""","""16/06/2025 20:00""","""- - -""","""- - -""","""- - -""","""- - -""","""- - -""",359,...,76,76,75,79,78,90,73,80,81,Biblioteconomia
2,316,"""- - -""","""- - -""","""16/06/2025 19:59""","""- - -""","""- - -""","""- - -""","""- - -""","""- - -""",358,...,63,63,61,59,60,83,74,73,76,Biblioteconomia
3,315,"""- - -""","""- - -""","""16/06/2025 19:57""","""- - -""","""- - -""","""- - -""","""- - -""","""- - -""",357,...,69,71,70,69,66,89,68,64,66,Biblioteconomia
4,314,"""- - -""","""- - -""","""16/06/2025 19:55""","""- - -""","""- - -""","""- - -""","""- - -""","""- - -""",356,...,71,89,63,68,87,88,89,90,91,Museologia


In [93]:
# Mostrar estrutura / schema do DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 61 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Identificador                        300 non-null    int64 
 1    "Codigo externo do registro"        300 non-null    object
 2    Etapa                               300 non-null    object
 3    "Data da criacao"                   300 non-null    object
 4    "Momento do ultimo ganho"           300 non-null    object
 5    "Momento da ultima perda"           300 non-null    object
 6    "Na mesma etapa desde"              300 non-null    object
 7    "Codigo - Oferta de curso"          300 non-null    object
 8    "Nome - Oferta de curso"            300 non-null    object
 9    "Identificador da pessoa"           300 non-null    int64 
 10   "Codigo externo da pessoa"          300 non-null    object
 11   "Nome da pessoa"                    300 non-

### Selecionar apenas dados necessários

In [94]:
colunas_desejadas = [
	"Matematica",
	'"Portugues "', 
	"Literatura",
	"Redacao",
	"Quimica",
	"Fisica",
	"Biologia",
	"Geografia",
	"Historia",
	"Filosofia",
	"Sociologia",
	"Artes",
	'"Area de Preferencia"', 
	"Curso"
]

# Remover espaços dos nomes das colunas
df.columns = df.columns.str.strip()

# Verificar nomes das colunas após strip
print(df.columns.tolist())


# Agora filtrar as colunas
df_filtrado = df[colunas_desejadas]
df_filtrado.info()

['Identificador', '"Codigo externo do registro"', 'Etapa', '"Data da criacao"', '"Momento do ultimo ganho"', '"Momento da ultima perda"', '"Na mesma etapa desde"', '"Codigo - Oferta de curso"', '"Nome - Oferta de curso"', '"Identificador da pessoa"', '"Codigo externo da pessoa"', '"Nome da pessoa"', '"Nome do responsavel"', 'Atividade', '"E-mail da pessoa"', '"E-mails secundarios"', '"Telefone da pessoa"', '"Telefones secundarios"', 'CPF', '"Data de nascimento"', '"Valor da oportunidade"', '"Data da atividade"', '"Forma de ingresso"', '"Grau de instrucao"', '"Escola de origem"', '"Nota do ENEM"', '"Processo seletivo"', 'Unidade', 'Modalidade', 'Status', '"Local da oferta"', 'Canal', '"Resumo atual"', 'Objecao', '"Observacoes da perda"', 'Endereco', 'Numero', 'Bairro', 'Cidade', 'UF', 'Concorrentes', 'Probabilidade', '"[AE] Situacao de matricula"', '"Area de Preferencia"', '"[AE] % de nota geral"', '"[AE] % de frequencia geral"', '"[AE] Satisfacao"', '"[AE] Qtd de lancamentos Vencidos"'

In [95]:
df_filtrado.head(5)

,Matematica,"""Portugues """,Literatura,Redacao,Quimica,Fisica,Biologia,Geografia,Historia,Filosofia,Sociologia,Artes,"""Area de Preferencia""",Curso
0,82,96,98,90,89,88,86,87,99,86,85,89,Humanas,Biblioteconomia
1,80,90,89,76,76,75,79,78,90,73,80,81,Humanas,Biblioteconomia
2,65,78,80,63,63,61,59,60,83,74,73,76,Humanas,Biblioteconomia
3,70,90,88,69,71,70,69,66,89,68,64,66,Humanas,Biblioteconomia
4,74,72,69,71,89,63,68,87,88,89,90,91,Humanas,Museologia


In [96]:
#  Medidas estatísticas do DF
df_filtrado.describe()

,Matematica,"""Portugues """,Literatura,Redacao,Quimica,Fisica,Biologia,Geografia,Historia,Filosofia,Sociologia,Artes
count,300.000000,300.000000,300.000000,300.000000,300.0000,300.00000,300.000000,300.000000,300.000000,300.000000,300.00000,300.000000
mean,80.233333,77.190000,77.546667,77.336667,78.4400,79.41000,79.600000,77.813333,78.776667,78.100000,78.35000,78.823333
std,11.578100,9.577735,9.072791,9.430435,10.1609,10.61402,10.555792,8.762864,8.803320,9.097943,9.98096,10.090168
min,58.000000,58.000000,59.000000,48.000000,52.0000,50.00000,43.000000,57.000000,60.000000,59.000000,41.00000,53.000000
25%,69.000000,68.000000,71.000000,70.000000,71.0000,72.00000,72.750000,72.000000,73.750000,73.000000,72.00000,71.000000
50%,80.000000,78.000000,78.000000,77.500000,79.0000,80.00000,80.000000,79.000000,79.000000,79.000000,79.00000,79.000000
75%,90.000000,85.000000,84.000000,84.000000,86.0000,88.25000,88.000000,82.000000,84.000000,84.000000,86.00000,87.000000
max,100.000000,99.000000,99.000000,99.000000,99.0000,99.00000,99.000000,98.000000,99.000000,98.000000,99.00000,99.000000


In [97]:
# Mapeando as áreas de cada curso
areas_cursos = {
    'Sociologia': 'Humanas',
    'Psicologia': 'Humanas',
    'Odontologia': 'Saúde',
    'Medicina Veterinária': 'Biológicas',
    'Medicina': 'Saúde',
    'Historia': 'Humanas',
    'Engenharia Eletrica': 'Exatas', 
    'Engenharia da Computação': 'Tecnologia',
    'Engenharia Civil': 'Exatas',
    'Enfermagem': 'Saúde',
    'Biologia': 'Biológicas',
    'Ciencia da Computação': 'Tecnologia',
    'Arquitetura e Urbanismo': 'Exatas',
    'Direito': 'Humanas',
    'Design': 'Exatas',
    'Artes Visuais': 'Artes',
    'Engenharia de Produção': 'Exatas',
    'Engenharia Mecânica': 'Exatas',
    'Engenharia Química': 'Exatas',
    'Engenharia Ambiental': 'Exatas',
    'Engenharia de Alimentos': 'Exatas',
    'Engenharia de Software': 'Tecnologia',
    'Geografia': 'Humanas',
    'Filosofia': 'Humanas',
    'Quimica': 'Exatas',
    'Fisica': 'Exatas',
    'Matematica': 'Exatas',
    'Letras': 'Linguagens',
    'Danca': 'Artes',
    'Musica': 'Artes',
    'Teatro': 'Artes',
    'Artes Cênicas': 'Artes',
    'Artes Plásticas': 'Artes',
    'Cinema': 'Artes',
    'Sistemas da Informação': 'Tecnologia',
    'Agronomia': 'Biológicas',
    'Historia': 'Humanas',
    'Ciencias Sociais': 'Humanas',
    'Servico Social': 'Humanas',
    'Educacao Fisica': 'Saúde',
    'Relacoes Internacionais': 'Humanas',
    'Jornalismo': 'Comunicacao',
    'Publicidade e Propaganda': 'Comunicacao',
    'Relações Públicas': 'Comunicacao',
    'Marketing': 'Comunicacao',
    'Administração': 'Humanas',
    'Farmacia': 'Biológicas',
    'Biomedicina': 'Biológicas',
    'Nutricao': 'Biológicas',
    'Fisioterapia': 'Saúde',
    'Fonoaudiologia': 'Saúde',
    'Analise e Desenvolvimento de Sistemas': 'Tecnologia',
    'Ciencia de Dados': 'Tecnologia',
    'Linguistica': 'Linguagens',
    'Pedagogia': 'Humanas',
    'Engenharia Florestal': 'Biologicas',
    'Moda': 'Artes',
    'Design Grafico': 'Artes',
    'Design de Interiores': 'Artes',
    'Museologia': 'Humanas',
    'Gestao Ambiental': 'Exatas',
    'Biblioteconomia': 'Humanas'
}

## Features e transformação dos dados

In [98]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import xgboost as xgb

# Features (X) = notas por disciplina
feature_cols = ['Matematica', '"Portugues "', 'Literatura', 'Redacao', 'Quimica', 'Fisica', 'Biologia', 'Geografia', 'Historia', 'Filosofia', 'Sociologia', 'Artes','"Area de Preferencia"']
X = df_filtrado[feature_cols].copy()

X_numeric = X.drop(columns=['"Area de Preferencia"'])
area_pref = X[['"Area de Preferencia"']]

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
area_pref_encoded = encoder.fit_transform(area_pref)
area_pref_encoded = area_pref_encoded.astype(float)
area_pref_encoded = area_pref_encoded * 5


X = np.concatenate([X_numeric.values, area_pref_encoded], axis=1)

# Rótulo (y) = curso que o aluno faz
y_categorico = df_filtrado["Curso"]

# Transformar a variável curso para formato numérico
label_curso = LabelEncoder()
y = label_curso.fit_transform(y_categorico)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)



## Treinamento com Random Forest

In [99]:
kf_rf = KFold(n_splits=5, shuffle=True, random_state=None)


y_labels_rf = []
y_predict_rf = []

fold_scores = []

rf_model = RandomForestClassifier(n_estimators=100, random_state=42, oob_score=True,max_depth=20,criterion='gini')
# Treinar o modelo Random Forest com validação cruzada
for train, test in kf_rf.split(X):
        
        X_train_fold, X_test_fold = X[train], X[test]
        y_train_fold, y_test_fold = y[train], y[test]

        #Treina o modelo com esse conjunto de treinamento
        rf_model.fit(X_train, y_train)
        
        # Predições
        rf_model.fit(X_train_fold, y_train_fold)
        y_test_pred = rf_model.predict(X_test_fold)

        y_labels_rf.extend(y_test_fold)
        y_predict_rf.extend(y_test_pred)
        fold_scores.append(accuracy_score(y_test_fold, y_test_pred))

# Avaliação do modelo
print("Avaliação do modelo Random Forest:")
print(f"Acurácia média: {np.mean(fold_scores):.4f}")
print(f"Acurácia mínima: {np.min(fold_scores):.4f}")
print(f"Acurácia máxima: {np.max(fold_scores):.4f}")
print("Relatório de Classificação:")
print(classification_report(
    y_labels_rf, y_predict_rf,
    target_names=label_curso.classes_,
    labels=np.arange(len(label_curso.classes_))
))
print("Matriz de Confusão:")
print(confusion_matrix(
    y_labels_rf, y_predict_rf,
    labels=np.arange(len(label_curso.classes_))
))

print(f"Score OOB (Out-of-Bag) do modelo: {rf_model.oob_score_:.4f}")

/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_pred = type_of_target(y_pred, input_name="y_pred")
/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the n

Avaliação do modelo Random Forest:
Acurácia média: 0.2767
Acurácia mínima: 0.2167
Acurácia máxima: 0.3333
Relatório de Classificação:
                           precision    recall  f1-score   support

                      ADS       0.29      0.40      0.33         5
            Administração       1.00      0.20      0.33         5
                Agronomia       0.27      0.60      0.38         5
  Arquitetura e Urbanismo       1.00      0.75      0.86         4
            Artes Cênicas       0.00      0.00      0.00         5
          Artes Plásticas       0.20      0.20      0.20         5
            Artes Visuais       0.00      0.00      0.00         5
          Biblioteconomia       0.25      0.20      0.22         5
                 Biologia       0.38      0.50      0.43         6
              Biomedicina       0.17      0.20      0.18         5
                   Cinema       0.33      0.40      0.36         5
    Ciência da Computação       0.00      0.00      0.00     

/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_pred = type_of_target(y_pred, input_name="y_pred")
/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metri

## Treinamento com XGBoost


In [100]:
kf_xgb = StratifiedKFold(n_splits=5, shuffle=True, random_state=None)

y_labels_xgb = []
y_predict_xgb = []

fold_scores = []


# Separa em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
   
# Criar DMatrix para XGBoost
dtrain_clf = xgb.DMatrix(X_train, y_train)
dtest_clf = xgb.DMatrix(X_test, y_test)

# Parametros do modelo XGBoost e número de iterações
params = {"objective": "multi:softprob", "tree_method": "hist", "num_class": len(np.unique(y))}
n = 1000

# Realizar validação cruzada com XGBoost
# Num_boost_round é o número de iterações
# Early stopping é usado para evitar overfitting - se não houver melhoria na métrica de validação por 50 iterações, o treinamento é interrompido
# Nfold é o número de divisões para validação cruzada
# Params são os parâmetros do modelo
# Metrics são as métricas a serem avaliadas durante a validação cruzada
xgb_model = xgb.XGBClassifier(
   objective="multi:softprob",
   tree_method="hist",
   num_class=len(np.unique(y)),
   n_estimators=n,
   eval_metric="mlogloss",
   verbosity=1,
   random_state=42,
)

# Treinar o modelo Random Forest com validação cruzada
for train, test in kf_xgb.split(X,y):
        
        X_train_fold, X_test_fold = X[train], X[test]
        y_train_fold, y_test_fold = y[train], y[test]

        #Treina o modelo com esse conjunto de treinamento
        xgb_model.fit(X_train, y_train)
        
        # Predições
        xgb_model.fit(X_train_fold, y_train_fold)
        y_test_pred = xgb_model.predict(X_test_fold)

        y_labels_xgb.extend(y_test_fold)
        y_predict_xgb.extend(y_test_pred)
        fold_scores.append(accuracy_score(y_test_fold, y_test_pred))


print("Avaliação do modelo XGBoost:")
print(f"Acurácia média: {np.mean(fold_scores):.4f}")
print(f"Acurácia mínima: {np.min(fold_scores):.4f}")
print(f"Acurácia máxima: {np.max(fold_scores):.4f}")
print("Relatório de Classificação:")
print(classification_report(
    y_labels_xgb, y_predict_xgb,
    target_names=label_curso.classes_,
    labels=np.arange(len(label_curso.classes_))
))
print("Matriz de Confusão:")
print(confusion_matrix(
    y_labels_xgb, y_predict_xgb,
    labels=np.arange(len(label_curso.classes_))
))

/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_pred = type_of_target(y_pred, input_name="y_pred")
/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = ty

Avaliação do modelo XGBoost:
Acurácia média: 0.3133
Acurácia mínima: 0.2500
Acurácia máxima: 0.4167
Relatório de Classificação:
                           precision    recall  f1-score   support

                      ADS       0.20      0.20      0.20         5
            Administração       0.67      0.40      0.50         5
                Agronomia       0.17      0.20      0.18         5
  Arquitetura e Urbanismo       0.50      0.50      0.50         4
            Artes Cênicas       0.17      0.20      0.18         5
          Artes Plásticas       0.25      0.20      0.22         5
            Artes Visuais       0.17      0.20      0.18         5
          Biblioteconomia       0.60      0.60      0.60         5
                 Biologia       0.22      0.33      0.27         6
              Biomedicina       0.14      0.20      0.17         5
                   Cinema       0.50      0.40      0.44         5
    Ciência da Computação       0.00      0.00      0.00         5


/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
/home/cecilia/.local/share/virtualenvs/NCJNet-E8OimlyP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_pred = type_of_target(y_pred, input_name="y_pred")


## Treinamento com SVM

## Salvar o modelo

In [101]:
import joblib 

# Salvar modelo
joblib.dump(rf_model, './model/sugestor.pkl')
# salvar encoder
joblib.dump(label_curso, './model/labelCurso.pkl')

['./model/labelCurso.pkl']

In [102]:
import joblib 

joblib.dump(xgb_model, './model/sugestorXGB.pkl')
joblib.dump(encoder, './model/encoderAreaPref.pkl')

['./model/encoderAreaPref.pkl']